# Data Exploration and Introduction

### import packages

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import pickle
import re
%matplotlib inline

### import helper functions


In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join('..', 'src')))

'''helper functions'''
import make_dataset as mk # make dataset - data processing

import make_dataset_PIMS_API as mkp 

[nltk_data] Downloading package stopwords to C:\Users\Jonas
[nltk_data]     Nothnagel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Jonas
[nltk_data]     Nothnagel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Import API data:

In [3]:
# uses neural translation for french and spanish logframes. Takes approx 4h withoug GPU support.
#df_api, df_logs = mk.import_api_data(all_timelines = True)

# Process Data

## Process taxonomy data

In [4]:
# use helper functios from make_dataset to import tagging table:
taxonomy = mk.import_raw_data('tagging_table_v17') 
print('')

# use helper function to import and process indicators:
indicators = mk.import_indicators()
print('')

# create datasets: processed data, dataframe with only raw texts and download data from PIMS API to compare and fill in:
taxonomy_processed, pims = mk.create_training_texts(taxonomy, compare_with_API = True, under_implementation = True, replace_short = True)

print('save data')
taxonomy_processed.to_excel(os.path.abspath(os.path.join('..', 'data/interim'))+'/taxonomy_processed.xlsx', index=False)

raw data shape: (655, 193)
only rows that have valid PIMS ID: (655, 193)
only columns with entries: (655, 192)


appending indicators...
done!
______________________________

extracting and merging all text fields - descriptions, objectives, outcomes, outputs, indicators...
done!
______________________________

(655, 201)
keep only projects that are currently under implementation...
(0, 201)
done!
______________________________

Pims_plus API is considered to complete training data....
Remove duplications from different time lines - delete exact otherwise keep latest...
done!
shape of data after removing timeline duplications:
0
______________________________

flag projects that were tagged by looking at project document...
number of documents with and without proper text and only titles both          0
right_only    0
left_only     0
Name: title_only, dtype: int64
done!
______________________________

Checking list of PIMS_ID that are not implemented or cancelled and remove from dataf

KeyError: "['title_y' 'all_text_and_title' 'all_text_and_title_y' 'output_1.1'\n 'output_1.2' 'output_1.3' 'output_1.4' 'output_1.5' 'outcome_2'\n 'output_2.1' 'output_2.2' 'output_2.3' 'output_2.4' 'output_2.5'\n 'outcome_3' 'output_3.1' 'output_3.2' 'output_3.3' 'output_3.4'\n 'output_3.5' 'outcome_4' 'output_4.1' 'output_4.2' 'output_4.3'\n 'output_4.4' 'output_4.5' 'outcome_5' 'output_5.1' 'output_5.2'\n 'output_5.3' 'output_5.4_(no_entry)' 'output_5.5_(no_entry)'\n 'outcome_6_(no_entry)' 'output_6.1' 'output_6.2' 'outcome_1' 'rta'\n 'project_objective' 'project_objective_2' 'project_description' 'taggers'] not found in axis"

### save list of PIMS that have no text in taxonomy:

In [5]:
taxonomy_no_pims, pims = mk.create_training_texts(taxonomy, compare_with_API = False, under_implementation = True)
taxonomy_filtered_no_pims, data = mk.create_subset(taxonomy_no_pims, "all_text")
print('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
print(taxonomy_no_pims.shape)
print(taxonomy_filtered_no_pims.shape)

appending indicators...
done!
______________________________

extracting and merging all text fields - descriptions, objectives, outcomes, outputs, indicators...
done!
______________________________

(655, 98)
keep only projects that are currently under implementation...
(655, 98)
done!
______________________________

only taxonomy data is considered
flag projects that were tagged by looking at project document...
number of documents with and without proper text and only titles False         490
True          165
right_only      0
Name: title_only, dtype: int64
done!
______________________________

Checking list of PIMS_ID that are not implemented or cancelled and remove from dataframa if no text exists...
Number of projects that will be removed: 0
______________________________

processing done!
final shape of data: (655, 60)
deleting all empty fields and creating subset for: all_text
___________
original data size is: 655
remaining projects with non empty field all_text (490, 60)
___

In [11]:
# taxonomy_no_pims['all_text'] = taxonomy_no_pims['all_text'].fillna('').astype(str)
# df1 =  taxonomy_no_pims[taxonomy_no_pims['all_text'].str.contains('[A-Za-z]')]

# total_merge = taxonomy_no_pims.merge(df1, on='PIMS_ID', how='outer', indicator=True)
# total_merge['all_text_y'] = total_merge['all_text_y'].fillna('').astype(str)

# R_1 = total_merge[total_merge['_merge']=='both']
# R_2 = total_merge[total_merge['_merge']=='left_only']
# R_3 = total_merge[total_merge['_merge']=='right_only']

##  detect missing and faulty logframes

In [12]:
##save flaged columns and texts:
# flag = taxonomy_processed[['PIMS_ID', 'all_text', 'description', 'pims_description', 'pims_objectives','pims_outcomes', 'pims_text','title_only']]
# flag = flag.loc[flag['title_only'] == True]
# flag = flag.reset_index(drop=True)
# flag = flag[['PIMS_ID','pims_text', 'title_only']]
# flag['pims_text'] = flag['pims_text'].fillna('not available')

# flag = flag.rename(columns={"title_only": "available_from_pims"})

# for index, row in flag.iterrows():
#     if row['pims_text'] == 'not available':
#         flag.at[index, 'available_from_pims'] = False
        
#     if len(row['pims_text']) < 600 and row['available_from_pims'] == True:
#         flag.at[index, 'available_but_incomplete'] = True
#     else:
#         flag.at[index, 'available_but_incomplete'] = False
           
# flag.to_excel(os.path.abspath(os.path.join('..', 'data/temp'))+'/missing_logframes.xlsx')    

# # not available or if available non useable list:
# flag_2 = flag
# for index, row in flag_2.iterrows():
    
#     if row['available_from_pims'] == True:
#         if row['available_but_incomplete'] == False:
#             flag_2.drop(index, inplace=True)
# flag_2 = flag_2.reset_index(drop=True) 
           
# flag_2.to_excel(os.path.abspath(os.path.join('..', 'data/temp'))+'/checkup.xlsx') 

# #no logs
# no_logs  = flag[flag.available_from_pims == False]
# no_logs = no_logs.reset_index(drop=True)

# no_logs = no_logs[['PIMS_ID', 'pims_text', 'available_from_pims']]

# no_logs.to_excel(os.path.abspath(os.path.join('..', 'data/temp'))+'/no_logframes.xlsx')

# #logs but seems incomplete:
# incomplete  = flag[flag.available_but_incomplete == True]
# incomplete = incomplete.reset_index(drop=True)

# incomplete = incomplete[['PIMS_ID', 'pims_text', 'available_but_incomplete']]

# incomplete.to_excel(os.path.abspath(os.path.join('..', 'data/temp'))+'/incomplete.xlsx')

### Focus on most commonly appearing text: 'all_text'

In [13]:
taxonomy_filtered, data = mk.create_subset(taxonomy_processed, "all_text")

print('save data...')
taxonomy_filtered.to_excel(os.path.abspath(os.path.join('..', 'data/interim'))+'/taxonomy_filtered.xlsx', index=False)

deleting all empty fields and creating subset for: all_text
___________
original data size is: 635
remaining projects with non empty field all_text (604, 65)
_________________________
average lenght of texts: 4382.806299212599
Median of texts: 3473.0
save data...


### check lenght of text columns for training data:

In [14]:
import plotly.express as px
all_text, data = mk.create_subset(taxonomy_filtered, "all_text")
all_text['all_text_len'] = all_text['all_text'].apply(len)
print('')
print('')
logframe, data = mk.create_subset(taxonomy_filtered, "logframe")
logframe['logframe_len'] = logframe['logframe'].apply(len) 
print('')
print('')
description, data = mk.create_subset(taxonomy_filtered, "description")
description['description_len'] = description['description'].apply(len)
print('')
print('')
objectives, data = mk.create_subset(taxonomy_filtered, "objectives")
objectives['objectives_len'] = objectives['objectives'].apply(len)
print('')
print('')
outcomes, data = mk.create_subset(taxonomy_filtered, "outcomes")
outcomes['outcomes_len'] = objectives['outcomes'].apply(len)
print('')
print('')
outputs, data = mk.create_subset(taxonomy_filtered, "outputs")
outputs['outcomes_len'] = outputs['outputs'].apply(len)
print('')
print('')
indicator, data = mk.create_subset(taxonomy_filtered, "Indicator")
indicator['indicator_len'] = outputs['Indicator'].apply(len)
print('')
print('')
pims_text, data = mk.create_subset(taxonomy_filtered, "pims_text")
pims_text['pims_text_len'] = pims_text['pims_text'].apply(len)

ModuleNotFoundError: No module named 'plotly'

In [ ]:
### description:
all_text.hist(column='all_text_len', bins=100)

In [ ]:
from matplotlib_venn import venn2
import matplotlib.pyplot as plt
fig = plt.figure()
a = all_text.PIMS_ID.to_list()
b = logframe.PIMS_ID.to_list()
c = description.PIMS_ID.to_list()
d = objectives.PIMS_ID.to_list()
e = outputs.PIMS_ID.to_list()
f = outcomes.PIMS_ID.to_list()

set1 = set(a)
set2 = set(b)
set3 = set(c)
set4 = set(d)
set5 = set(e)
set6 = set(f)

v = venn2([set1, set6], ('all_text', 'outcomes'))

In [ ]:
taxonomy_filtered.columns

In [ ]:

labels_only = pd.DataFrame(taxonomy_filtered, columns=[ 'technical_team', 'cross-cutting_programme_1', 'technical_area_1',
       'technical_sub-area_1', 'cross-cutting_programme_2', 'technical_area_2',
       'technical_sub-area_2', 'transformed_sectors',
       'landscapes_1', 'sub-landscapes_1', 'landscapes_2', 'sub-landscapes_2',
       'landscapes_3', 'sub-landscapes_3', 'sdg_goal_reference_from_pims+',
       'sdg_goal', 'sdg_targets_(top_three)', 'sdg_goal_summary', 'undp_roles',
       'main_theme_', 'sub-_strategy_1', 'main_strategy_2', 'sub-_strategy_2',
       'main_strategy_3', 'sub-_strategy_3', 'pathways', 'targeted_risks',
       '*conventions/protocols/plans', '*social_inclusion_&_engagement',
       '*gender_equality', '*types_of_private_sector', '*hot_topics'])

multi_class = pd.DataFrame(taxonomy_filtered, columns=['technical_team'])

multi_label_upper_3 = pd.DataFrame(taxonomy_filtered, columns=['undp_roles', 'pathways',
       'targeted_risks', '*conventions/protocols/plans',
       '*social_inclusion_&_engagement', '*gender_equality',
       '*types_of_private_sector', '*hot_topics'])


sequential_multi_class_multi_label = pd.DataFrame(taxonomy_filtered, columns=['main_strategy_1',  'main_strategy_2', 'main_strategy_3',  'landscapes_1','landscapes_2', 'landscapes_3']) 

multi_label_hierarchical_upper_2 = pd.DataFrame(taxonomy_filtered, columns=['technical_sub-area_1', 'technical_sub-area_2']) 

multi_label_hierarchical_upper_3 = pd.DataFrame(taxonomy_filtered, columns=['technical_sub-sub_area_1', 'technical_sub-sub_area_2', 
                                                                              'sub-landscapes', 'sub-landscapes_2', 'sub-landscapes_3',  
                                                                             'sub-_strategy_1', 'sub-_strategy_2', 'sub-_strategy_3']) 

multi_label_few_hierarchical = pd.DataFrame(taxonomy_filtered, columns=['transformed_sectors']) 

In [ ]:
for index, row in taxonomy_filtered.iterrows():
    
    if row['technical_team'] == 'Climate_Change_Mitigation_Programme':
        print(row['PIMS_ID'])

## Multi-Class Categorisation: Categorical charts

In [ ]:
for cate in multi_label_upper_3:
    class_ = vis.disp_category(cate, taxonomy_filtered, min_items=1)

## Sequential multi label/multi class: Categorical charts


In [ ]:
for cate in sequential_multi_class_multi_label:
    print (cate)
    class_ = vis.plotly_pie(cate, taxonomy_filtered, min_items=1)

## Radar chart

In [ ]:
for cate in multi_class:
    print (cate)
    class_ = vis.plotly_radar(cate, taxonomy_filtered, min_items=1)

# Summary:

* Very small training data set.
* Highly skewed distribution of training data set. 
* Highly unbalanced classification problems. 
* Large number of labels to predict.
* Hierarchical structures.
* Additional rules - "only up to two/three categories".
